In [ ]:
from simulator_v2.cartpole_torch.config import SystemConfiguration
from simulator_v2.cartpole_torch.system import CartPoleSystem
from simulator_v2.cartpole_torch.state import State

import numpy as np
import numpy.typing as npt
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

## About the simulation

Our simulation's state is represented via 4 numbers:
- $x$ - cart position on the rail (default value is 0).
- $\theta$ - pole angle in radians. The default value is 0, when the pole is hanging down. Always belongs to $[0, 2\pi)$
- $\dot{x}$ - cart velocity (can be negative, if moving left).
- $\dot{\theta}$ - pole angular velocity (in radians, positive if moving counterclockwise).

## Pendulum (simple case)

Let us consider a system with no input first.
In other words, just a simple pendulum.

In [ ]:
system = CartPoleSystem()
system.reset_to_state(
    config=SystemConfiguration(),
    state=State(
        cart_position=0,
        pole_angle=1.23,
        cart_velocity=0,
        angular_velocity=0,
    ),
)
system.set_target(0.0)

In [ ]:
%timeit
SECONDS_TO_SIMULATE = 5
INPUT_DELTA = 0.03

SIM_STEPS = int(SECONDS_TO_SIMULATE / INPUT_DELTA)

history = np.empty(shape=(SIM_STEPS, 6))


for i in range(SIM_STEPS):
    state = system.get_state().as_tensor()

    history[i, 0] = system.timestamp()
    history[i, 1:5] = state
    history[i, 5] = 0.0

    system.advance(delta=INPUT_DELTA)

At this point, we have simulated 5 seconds of pendulum swinging.
Let us use Pandas DataFrames to explore the data we got from the simulation.

In [ ]:
def history_to_pandas(history: npt.NDArray) -> pd.DataFrame:
    """
    Converts history array to pandas DataFrame

    Args:
        history (npt.NDArray): Nx6 array of floats

    Returns:
        pd.DataFrame: Same array but dataframe
    """
    COLUMNS = [
        "timestamp",
        "cart_position",
        "pole_angle",
        "cart_velocity",
        "angular_velocity",
        "acceleration",
    ]
    return pd.DataFrame(
        {column_name: history[:, ind] for ind, column_name in enumerate(COLUMNS)}
    )

And let us also define a function to calculate energies of our system for each point of time

In [ ]:
def add_energies(history: pd.DataFrame) -> pd.DataFrame:
    """
    Calculates energies and returns a copy of dataframe with 4 extra columns:
    - cart kinetic energy
    - pole kinetic energy
    - pole potential energy
    - total energy

    Args:
        history (pd.DataFrame)

    Returns:
        pd.DataFrame
    """
    cart_mass = system._config.parameters.cart_mass or 0.2
    cart_mass = 0.2
    pole_mass = system._config.parameters.pole_mass
    pole_len = system._config.parameters.pole_length
    grav = system._config.parameters.gravity

    v = history["cart_velocity"]
    ang = history["pole_angle"]
    thetadot = history["angular_velocity"]

    kin_cart = cart_mass * (v**2) / 2  # type: ignore
    pot_pole = pole_mass * grav * pole_len / 2 * (1 - np.cos(ang))

    kin_pole = (pole_mass / 2) * (
        v**2
        + ((pole_len**2) * (thetadot**2)) / 3
        + (pole_len * v * thetadot * np.cos(ang))
    )

    energies = {
        "kinetic_cart": kin_cart,
        "kinetic_pole": kin_pole,
        "potential_pole": pot_pole,
        "total_energy": kin_cart + kin_pole + pot_pole,
    }

    return history.assign(**energies)

In [ ]:
history = history_to_pandas(history)
history = add_energies(history)
history.head()

Now lets plot some graphs

In [ ]:
from math import ceil


PLOT_COLUMNS = [
    "pole_angle",
    "angular_velocity",
    "kinetic_pole",
    "potential_pole",
    "total_energy",
]

nrows = round(np.sqrt(len(PLOT_COLUMNS)))
ncols = ceil(len(PLOT_COLUMNS) / nrows)


fig, ax = plt.subplots(
    nrows=nrows,
    ncols=ncols,
    figsize=(8 * ncols, 8 * nrows),
)


for i, column in enumerate(PLOT_COLUMNS):
    cur_ax = ax[i // ncols][i % ncols]
    sns.lineplot(
        data=history,
        x="timestamp",
        y=column,
        ax=cur_ax,
    )


sns.lineplot(
    data=history,
    x="timestamp",
    y=np.round(history["total_energy"], 5),
    ax=ax[1, 2],
    label="total energy (rounded to 5th digit)",
)

On the second graph from the second row one migth see that `total_energy` seems to be increasing over time.
Please, note the scale of the graph - the difference is not that dramatic.
We included the last graph to show that: it contains `total_energy`, but rounded to 5-th digit after the dot.
One might see that the total energy is indeed constant.

## CartPole (more complex case)

This time we will make use of controlling the system.
To be precise, we will introduce cart acceleration.

In [ ]:
system = CartPoleSystem()
system.reset_to_state(
    config=SystemConfiguration(),
    state=State.home(),
)

In [ ]:
%timeit
SECONDS_TO_SIMULATE = 15
INPUT_DELTA = 0.03

SIM_STEPS = int(SECONDS_TO_SIMULATE / INPUT_DELTA)

history = np.empty(shape=(SIM_STEPS, 6))

for i in range(SIM_STEPS):
    # This time we calculate the acceleration of the cart using some weird formula
    k = int((i % 100) < 50)
    acc = (-1)**k * i / SIM_STEPS
    system.set_target(acc)

    state = system.get_state().as_tensor()

    history[i, 0] = system.timestamp()
    history[i, 1:5] = state
    history[i, 5] = acc

    system.advance(delta=INPUT_DELTA)

In [ ]:
history = history_to_pandas(history)
history = add_energies(history)
history.head()

In [ ]:
PLOT_COLUMNS = [
    "cart_position",
    "pole_angle",
    "cart_velocity",
    "angular_velocity",
    "kinetic_cart",
    "kinetic_pole",
    "potential_pole",
    "total_energy",
]

nrows = round(np.sqrt(len(PLOT_COLUMNS)))
ncols = ceil(len(PLOT_COLUMNS) / nrows)

fig, ax = plt.subplots(
    nrows=nrows,
    ncols=ncols,
    figsize=(8 * ncols, 8 * nrows),
)

for i, column in enumerate(PLOT_COLUMNS):
    cur_ax = ax[i // ncols][i % ncols]
    sns.lineplot(
        data=history,
        x="timestamp",
        y=column,
        ax=cur_ax,
        label=column,
    )
    sns.lineplot(
        data=history,
        x="timestamp",
        y="acceleration",
        ax=cur_ax,
        label="acceleration",
    )

angles_fix_mask = history["pole_angle"] > 5.5
fixed_angles = history["pole_angle"].copy(True)
fixed_angles[angles_fix_mask] -= 2 * np.pi
sns.lineplot(
    data=history,
    x="timestamp",
    y=fixed_angles,
    ax=ax[2][2],
)
sns.lineplot(
    data=history,
    x="timestamp",
    y="acceleration",
    ax=ax[2][2],
    label="acceleration",
)

In [ ]:
g = sns.lineplot(
    data=history,
    x="timestamp",
    y=np.abs(fixed_angles),
    label="angle (radians)",
)
g.set_ylabel("")
sns.lineplot(
    data=history,
    x="timestamp",
    y=history["potential_pole"] * 10,
    label="potential energy (scaled)",
)
g.set_ylabel("");